In [1]:
GENIUS_API_TOKEN = "owmqKAzXnbdUGxMUao4jqbIg3tDFXHvQBm7eHWYTeH8HeB5LcWutHto7kaugLokj"
CLIENT_ID = "9orlfIxfgWHnYqyD6oEkUBK8Kbi1ZsthuIAFh06G8f1kilCTufXNXcFrBRCaCjDA"
CLIENT_SECRET = "LaJt_TdRMs4LToYVnziOnBsxjKjoQzNRQGYc8zBVqGwhuQaCGECwYwq1zJ0DjCJP-dH0v6Gtb-HaMxIiyt2HKw"



In [2]:
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re

import csv
import time
import random



In [3]:
#make list of proxies and user-agents
with open("proxylist.txt","r") as proxy_file:
    proxy_list = proxy_file.read().split("\n")

with open("UserAgents.txt","r") as user_file:
    user_agent_list = user_file.read().split("\n")



In [4]:
#returns a request header with a random user-agent, useful for making lots of requests
def get_header(authorization=True):

    UserAgent = user_agent_list[random.randint(0,len(user_agent_list) - 1)]


    header = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
        "cache-control": "max-age=0",
        "dnt": "1",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "Authorization" : 'Bearer ' + GENIUS_API_TOKEN,
        "client_id": CLIENT_ID,
        "client_secret" : CLIENT_SECRET,
        "user-agent": UserAgent.rstrip()}

    if not authorization:
        header = {
            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "accept-encoding": "gzip, deflate, br",
            "accept-language": "en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
            "cache-control": "max-age=0",
            "dnt": "1",
            "sec-fetch-dest": "document",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "none",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1",
            "user-agent": UserAgent.rstrip()}

    return header

#get a random proxy
def get_proxy():
    p = proxy_list[random.randint(0,len(proxy_list) - 1)]
    return {"http" : p, "https": p}



    

In [16]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    
    #putting in too many requests make genius stop sending me data
    time.sleep(random.randint(20,30))

    response = requests.get(search_url, data=data, headers=get_header())
    print(f"artist info request status code for {artist_name}: {response.status_code}")
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    response = request_artist_info(artist_name, page)
    for i in range(song_cap):
        
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs

#request_song_url("Angry Samoans",1)

In [17]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    #putting in too many requests make genius stop sending me data
    time.sleep(random.randint(1,10))
    

    page = requests.get(url, headers=get_header(authorization=False))


    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='Lyrics__Container-sc-1ynbvzw-5 Dzxov').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #split up words that were joined together ex. "nowDiamonds" -> "now Diamonds"
    lyrics = re.sub(r'(\w)([A-Z])', r'\1 \2', lyrics)
    lyrics = re.sub(r'[\.!?](?=\s|$)|(?<=\w)[?]', ' ', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics
# DEMO
#print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

In [18]:
def write_lyrics_to_file(artist_name, song_count, genre=""):


    urls = request_song_url(artist_name, song_count)
    count = 0

    if len(urls) > 0:
        print("Writing to file\n")
        f = open('lyrics/' + artist_name.lower() + genre + '.txt', 'w+')
        for url in urls:
            try:
                lyrics = scrape_song_lyrics(url)
                f.write(lyrics)
                count += 1
            except:
                print("Problem scraping this url, trying next one")
                pass

        f.close()


        #num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + genre + '.txt', 'rb'))
        #print('Wrote {} lines to file from {} songs\n'.format(num_lines, song_count))
    else:
        print(f"Cannot find any songs by {artist_name}\n")

        #this is to test if we got blocked, if so wait a while

        #there should always be a beatles song
        beatles = request_song_url("Beatles", 1)

        #if there isnt one, the we are probably blocked 
        if len(beatles) == 0:
            print("sleeping for 5 min\n")
            time.sleep(300)

            #set count to -1 as a flag that we should try this one again

    
    return count
  
#write_lyrics_to_file("Beatles",1)

In [20]:
def read_txt(filename):
    #reads everyline from a .txt file and returns a single string object
    #with all of the text in the file
    with open(filename, 'r') as file:
        text = file.read()
    return text



In [21]:
folder_path = "artists/"


with open("lyricdata.csv", 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)

    #go through all the files in the artists folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if os.path.isfile(file_path):

            #open the file, and for each line, which is a name of an artist, get lyrics data
            with open(file_path, 'r') as file:
                for line in file:
                    
                    #some of the names have something like: Agent Orange (band), this removes that
                    name = re.sub(r'\(.*?\)', '', line).strip()
                    genre = os.path.splitext(filename)[0]
                    
                    #im choosing to search for 10 songs because I doubt there will be that much data available for every artist, so I don't want some, (like the beatles), with like 100 songs, and some with only 10, so im limiting it at 10. I am also doing this becuase idk how long this task will take my computer.
                    num_songs = write_lyrics_to_file(name, 10, genre=genre)


                    
                    if num_songs > 0:
                        #removes any char that would not be allowed in a filename
                        name = re.sub(r'[<>:"/\\|?*\x00-\x1F]', '', name)

                        #get one big string of lyrics to put in csv file
                        l = read_txt('lyrics/' + name.lower() + genre + '.txt')

                        #this is so i can fill lyricsdata.csv
                        row = [name, genre, num_songs, l]
                        writer.writerow(row)

                    #this means we got blocked, waited 5 min, now we should try again
                    elif num_songs == -1:
                        num_songs = write_lyrics_to_file(name, 10, genre=genre)

                        if num_songs > 0:
                            #removes any char that would not be allowed in a filename
                            name = re.sub(r'[<>:"/\\|?*\x00-\x1F]', '', name)
                            #get one big string of lyrics to put in csv file
                            l = read_txt('lyrics/' + name.lower() + genre + '.txt')

                            #this is so i can fill lyricsdata.csv
                            row = [name, genre, num_songs, l]
                            writer.writerow(row)



         
                 




artist info request status code for Amebix: 200
Found 10 songs by Amebix
Writing to file

Problem scraping this url, trying next one
artist info request status code for Angelic Upstarts: 200
Found 10 songs by Angelic Upstarts
Writing to file

artist info request status code for Angry Samoans: 200
Found 10 songs by Angry Samoans
Writing to file

artist info request status code for Anti-Nowhere League: 200
Found 10 songs by Anti-Nowhere League
Writing to file

artist info request status code for Anti-Pasti: 200
Found 10 songs by Anti-Pasti
Writing to file

artist info request status code for Au Pairs: 200
Found 10 songs by Au Pairs
Writing to file

artist info request status code for The Avengers: 200
Found 10 songs by The Avengers
Writing to file

artist info request status code for Art Attacks: 200
Found 10 songs by Art Attacks
Writing to file

artist info request status code for Bad Brains: 200
Found 10 songs by Bad Brains
Writing to file

artist info request status code for Bags: 200

OSError: [Errno 22] Invalid argument: 'lyrics/? and the mysterians1970s punk rock musicians.txt'

In [165]:
import pandas as pd


# new_rows = [["hi","hi",5, "lyrics"],["hi","hi",5, "lyrics"]]
# with open("lyricdata.csv", 'a', newline='') as file:
#     writer = csv.writer(file)
#     for row in new_rows:
        
#         writer.writerow(row)

df = pd.read_csv("lyricdata.csv")
df


,artist,genre,numsongs,lyrics
0,45 Grave,1970s punk rock musicians,10,"Pity the poor little darling now Oh, she never..."
1,Abrasive Wheels,1970s punk rock musicians,10,Got my marching orders in the morning post Whe...
2,The Adicts,1970s punk rock musicians,10,"Walk on, Walk on With Hope, in your hearts, an..."
3,Adolescents,1970s punk rock musicians,10,Amoeba Amoeba Amoeba Amoeba Amoeba Amoeba Amoe...
4,The Adverts,1970s punk rock musicians,10,"Life's short, don't make a mess of it To the e..."
5,Agent Orange,1970s punk rock musicians,10,They can make things worse for me Sometimes I'...
6,Alex Harvey,1970s punk rock musicians,10,N E X T Naked as sin an army towel covering...
7,The Alley Cats,1970s punk rock musicians,10,"Puddin' n' tain, puddin' n' tain Ask me again ..."
8,Alternative TV,1970s punk rock musicians,10,"Action, time and vision Action, time and visio..."
